In [6]:
import pandas as pd
import requests
import json
# Read the file using pandas
df = pd.read_stata('women2018ok.dta')
#put all city and countries into lists
citylist = df['cityname'].unique().tolist()
countylist= df['countyname'].unique().tolist()
locations = df[['cityname', 'countyname']].drop_duplicates().reset_index(drop=True)

# 接口地址
url = "https://api.map.baidu.com/place/v2/search"
# reques for response
key="o65PXhJ1yPA4oPf6JUgtjsNy12yes7AJ"
query="幼儿园"
tag="幼儿园"
region="海淀区"

KinderGartenNum = {}

params = {
    "query":    query,
    "tag":    tag,
    "region":    region,
    "output":    "json",
    "ak":       key,
    "page_size":    20,
    }
response = requests.get(url=url, params=params).json()
results = response["results"]


In [7]:

# Loop through the locations
for region in countylist:
    params = {
        "query":    query,
        "tag":    tag,
        "region":    region,
        "output":    "json",
        "ak":       key,
        "page_size":    20,
    }
    response = requests.get(url=url, params=params).json()
    count=response["total"]
    KinderGartenNum[region] = count

    



In [8]:
df['KinderGartenCount'] = df['countyname'].map(KinderGartenNum)




In [10]:
df.to_csv('output.csv', index=False)


In [14]:
df.to_stata('output.dta', encoding='utf-8')


UnicodeEncodeError: 'latin-1' codec can't encode characters in position 0-2: ordinal not in range(256)